# step1. 安裝CKIP

In [1]:
#!pip install -U ckiptagger[tfgpu,gdown]

In [1]:
# import os
# from pathlib import Path
# from collections import Counter
# import re
# import pickle
# from string import punctuation
# import datetime

# from gensim.models.word2vec import Word2Vec
# import numpy as np
# import pandas as pd
# from ckiptagger import data_utils, construct_dictionary, WS
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
# #from google.colab import drive

In [1]:
from pathlib import Path

import pandas as pd

from model_building import data_cleansing, get_punctuation, word_cut, word2vec_modle

# step2. 設定 GPU 加速斷詞速度。執行階段 > 變更執行階段類型

In [2]:
ckip_path = '/Users/zoe/Desktop/20201107_d4sg_corpus_mapping/data/ckip_model/data'
root_path = Path('/Users/zoe/Desktop/20201107_d4sg_corpus_mapping')
raw_df = pd.read_excel(root_path / 'data/200801至202008缺失類型(法規分段例).xlsx', sheet_name='法規')
parsed_column = '缺失內容'

#data_utils.download_data_gdown(ckip_path)
#ws = WS(ckip_path)



#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#ws = WS("./data", disable_cuda=False)

# step3. 將檔案路徑設定到自己的Google Drive

In [3]:
#drive.mount('/content/drive', force_remount=True)
!pwd

/Users/zoe/Desktop/20201107_d4sg_corpus_mapping


In [5]:
#採用包含法規50, 錯誤態樣50的法規, 事實, 建議拆分的版本


In [10]:
#word2vec_modle(step3_file, w2v_model_file)

In [11]:
def recommend_law(text, model_var_file):
    def text_preprocess(text):
        rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
        text = rule.sub(' ',str(text))
        text = re.sub(' +', '',text)
        text = ws([text], sentence_segmentation=True)
        text = [x for l in text for x in l]
        return text
    w2v_vocab, w2v_model, tfidf_feature, tfidf_text_vect, dictionary, df = pickle.load(open(model_var_file, 'rb'))
    text = text_preprocess(text)
    text_vect = np.zeros(100) # w2v size
    weight_sum = 0
    for word in text:
        if word in w2v_vocab and word in tfidf_feature:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    #binding all doc vector and add one input vector(seems extend would be better)
    tmp_vect = [*tfidf_text_vect, text_vect]
    new_cos_sim = cosine_similarity(tmp_vect, tmp_vect)
    sim_score = np.sort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:11]
    
    tmp_top_10_law = df[['法規名稱','條','缺失內容']].iloc[np.argsort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:11]]
    tmp_top_10_law['similarity_score'] = [round(score*100,1) for score in sim_score]
    return tmp_top_10_law

In [27]:
starttime = datetime.datetime.now()

newtext = '開標時有作拒絕往來廠商調查'
result = recommend_law(newtext, w2v_model_file)

# calculate running time
endtime = datetime.datetime.now()
print("搜尋推薦時間: ",endtime - starttime)

搜尋推薦時間:  0:00:01.383325


In [29]:
result

,法規名稱,條,缺失內容,similarity_score
13476,政府採購法,103,項次16─依政府採購法第103條第1項，經刊登政府採購公報之拒絕往來廠商自刊登次日起1年或3...,96.5
11178,政府採購法,103,依政府採購法第103條第1項，經刊登政府採購公報之拒絕往來廠商自刊登次日起1年或3年不得3加...,95.2
13171,政府採購法,103,依政府採購法第103條第1項，經刊登政府採購公報之拒絕往來廠商自刊登次日起1年或3年不得3加...,95.2
2976,政府採購法,50,本案允許「共同投標」，開標前已查詢拒絕往來廠商名單，惟未於開標後對得標廠商之共同投標成員長泉...,93.1
12739,政府採購法,50,依政府採購法第103條第1項，經刊登政府採購公報之拒絕往來廠商自刊登次日起1年或3年不得3加...,93.1
4629,政府採購法,50,政府採購法第50條第1項明定投標廠商有第103條第1項不得3加投標或作為決標對象之情形，經機...,93.0
10749,政府採購法,50,有關本案「查詢拒絕往來廠商名單」1節，查有缺失如下，請檢討改進： A.依政府採購法第50條第...,92.4
9393,政府採購法,50,本案受稽核文件附有貴局查詢各投標廠商是否為拒絕往來廠商名單，其中「台灣野村總研諮詢顧問股份有...,92.2
10222,政府採購法,50,本案開標日為103年2月21日，惟稽核文件查詢拒絕往來廠商名單時間為103年3月18日，顯未...,92.1
6752,政府採購法,50,政府採購法第50條第1項第6款明定，投標廠商有第103條第1項不得3加投標或作為決標對象之情...,91.9


In [ ]:
#bad formate inner open
#seems can't be last line of pickle dump

In [3]:
#step1 + data cleansing and transfer
dict_path = root_path / 'dictionary'
legal_name_file = dict_path / 'name_of_legal.txt'
word_file = dict_path / 'words.txt'
punc_file = dict_path / 'punctuation.pkl'

output_path = root_path / 'output'
step1_file = output_path / 'data_etl_step1.csv'
step2_file = output_path / 'data_etl_step2_noPunc.csv'
step3_file = output_path / 'data_etl_step3_noPuncDict.csv'
w2v_model_file = output_path / 'w2v_var.pkl'

In [4]:
data_cleansing(raw_df, output_file=step1_file, text_column=parsed_column)
get_punctuation(step1_file, output_file=punc_file, text_column=parsed_column)
word_cut(step1_file, step3_file, punc_file, parsed_column, legal_name_file, word_file, ckip_path)
#word2vec_modle(step3_file, w2v_model_file)

/Users/zoe/Desktop/20201107_d4sg_corpus_mapping/output/data_etl_step1.csv  exported.
/Users/zoe/Desktop/20201107_d4sg_corpus_mapping/dictionary/punctuation.pkl  exported.
/Users/zoe/Desktop/20201107_d4sg_corpus_mapping/output/data_etl_step3_noPuncDict.csv  exported.


In [5]:
parsed_column

'缺失內容'

In [11]:
import sys

In [13]:
sys.executable

'/Users/zoe/anaconda3/bin/python'